<a href="https://colab.research.google.com/github/TheJabbar/JamAPP/blob/master/FlairmodelSampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
import pandas as pd
link = 'https://drive.google.com/open?id=1ARSZM0gjAmryqyrRJff-Tkbn_CR1VxJ3'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('corpus_rev_cleaned_sample.xlsx')

link = 'https://drive.google.com/open?id=19viDsoa9J_Gn46RuVKQrpD8q4wBlejWg'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('stopwordsid.txt')

df = pd.read_excel('corpus_rev_cleaned_sample.xlsx')

In [2]:
!pip install --upgrade git+https://github.com/zalandoresearch/flair.git

  Cloning https://github.com/zalandoresearch/flair.git to /tmp/pip-req-build-ziajo1ao
  Running command git clone -q https://github.com/zalandoresearch/flair.git /tmp/pip-req-build-ziajo1ao
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 655kB 35.5MB/s 
     |████████████████████████████████| 798kB 47.6MB/s 
     |████████████████████████████████| 133kB 47.1MB/s 
     |████████████████████████████████| 1.0MB 51.1MB/s 
     |████████████████████████████████| 1.0MB 50.9MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-s7lkfavx/wheels/6a/78/0f/399330241d3bc69458cc4fe320dcdfbf818f9887803f0294e7
Successfully built flair
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Stored in directory: /root/.cache/pip

In [17]:
import pandas as pd
import numpy as np
import re
f = open("stopwordsid.txt", 'r')
stop_words = f.readlines()
f.close()
def removestopWords(s):
    s = ' '.join(word for word in s.split() if word not in stop_words)
    return s

df = pd.read_excel('corpus_rev_cleaned_sample.xlsx')
df = df.drop(['publish_date','doc_id','sentence_num','sentence_prev','sentence_next'],axis=1)
df['sentence'] = df['sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
df['sentence'] = df['sentence'].apply(lambda x : " ".join(re.findall('[\w]+',x)))
df['sentence'] = df['sentence'].apply(lambda x: removestopWords(x))
df = df.dropna()
df = df.reset_index(drop=True)
df['credibility'] = np.where(((df['formulasi']=='-')&(df['efektivitas']=='-')&(df['koordinasi']=='-')&(df['stance komunikasi']=='-')),0,1)
df.head()

,sentence,formulasi,efektivitas,koordinasi,stance komunikasi,credibility
0,kalau bisa menekan suku bunga kredit lebih ren...,-,-,-,-,0
1,perbankan masih menggunakan suku bunga lama se...,-,1,-,-,1
2,penurunan bi rate juga bisa membuat pergerakan...,-,1,-,-,1
3,kebijakan suku bunga ditempuh oleh bi merupaka...,1,-,-,-,1
4,penyesuaian bi rate akan memperkuat operasi mo...,1,1,-,-,1


In [0]:
link = 'https://drive.google.com/open?id=1o3SbepNJm3cCNkkF_7ROuwuSMyhpheL0'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('test_sample.csv')

In [25]:
dftest = pd.read_csv('test_sample.csv')
dftest['credibility'] = np.where(((dftest['formulasi']=='-')&(dftest['efektivitas']=='-')&(dftest['koordinasi']=='-')&(dftest['stance komunikasi']=='-')),0,1)
print(len(df))
len(dftest)


3674


502

In [30]:
import os
import random
os.remove("train.txt")
os.remove("test.txt")
os.remove("dev.txt")
import math
listrand = list(range(100))
random.shuffle(listrand)
listranddev = []
listrandtest = []
for i in range(math.floor(len(listrand)*0.2)):
    listranddev.append(listrand.pop())
file1 = open("train.txt","a", encoding="utf-8")
for i in listrand:
    file1.writelines(['__label__',str(df['credibility'].iloc[i]),' ',str(df['sentence'].iloc[i]),'\n'])
file1.close()
file2 = open("dev.txt","a", encoding="utf-8")
for i in listranddev:
    file2.writelines(['__label__',str(df['credibility'].iloc[i]),' ',str(df['sentence'].iloc[i]),'\n'])
file2.close()
file3 = open("test.txt","a", encoding="utf-8")
for i in range(len(dftest)):
    file3.writelines(['__label__',str(dftest['credibility'].iloc[i]),' ',str(dftest['sentence'].iloc[i]),'\n'])
file3.close()

[27, 12, 26, 45, 92, 22, 80, 66, 59, 95]


In [50]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.txt', dev_file='dev.txt', train_file='train.txt')

word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('id-forward'), FlairEmbeddings('id-backward')]

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=256, bidirectional=True,rnn_type='lstm',rnn_layers=1)

classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)

trainer = ModelTrainer(classifier, corpus)

trainer.train('./', max_epochs=30,checkpoint=True)
!mv best-model.pt best-cred-model.pt

2019-07-17 13:56:46,627 Reading data from .
2019-07-17 13:56:46,630 Train: train.txt
2019-07-17 13:56:46,631 Dev: dev.txt
2019-07-17 13:56:46,635 Test: test.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  import sys
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:463: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since versi

2019-07-17 13:56:50,251 Computing label dictionary. Progress:


100%|██████████| 80/80 [00:00<00:00, 73632.72it/s]

2019-07-17 13:56:50,259 [b'1', b'0']


2019-07-17 13:56:50,485 ----------------------------------------------------------------------------------------------------
2019-07-17 13:56:50,487 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.1)
          (encoder): Embedding(8823, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=8823, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.1)
          (encoder): Embedding(8823, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=8823, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=4196, out_features=4196, bias=True)
    (rnn): GRU(4196, 256, bidirectional=True)
    (dro

{'dev_loss_history': [tensor(0.9420, device='cuda:0'),
  tensor(0.7819, device='cuda:0'),
  tensor(1.1844, device='cuda:0'),
  tensor(0.7642, device='cuda:0'),
  tensor(0.7939, device='cuda:0'),
  tensor(0.8911, device='cuda:0'),
  tensor(0.8824, device='cuda:0'),
  tensor(0.9371, device='cuda:0'),
  tensor(0.9462, device='cuda:0'),
  tensor(0.8466, device='cuda:0'),
  tensor(0.8569, device='cuda:0'),
  tensor(0.8767, device='cuda:0'),
  tensor(0.8545, device='cuda:0'),
  tensor(0.8665, device='cuda:0'),
  tensor(0.8890, device='cuda:0'),
  tensor(0.8935, device='cuda:0'),
  tensor(0.8994, device='cuda:0'),
  tensor(0.9015, device='cuda:0'),
  tensor(0.9066, device='cuda:0'),
  tensor(0.9166, device='cuda:0'),
  tensor(0.9184, device='cuda:0'),
  tensor(0.9209, device='cuda:0'),
  tensor(0.9236, device='cuda:0'),
  tensor(0.9256, device='cuda:0'),
  tensor(0.9219, device='cuda:0'),
  tensor(0.9239, device='cuda:0'),
  tensor(0.9228, device='cuda:0'),
  tensor(0.9234, device='cuda:0'),


In [52]:
!ls

adc.json			devkomm.txt	 testeff.txt	  traineff.txt
best-cred-model.pt		devkoor.txt	 testform.txt	  trainform.txt
checkpoint.pt			dev.txt		 testkomm.txt	  training.log
corpus_rev_cleaned_sample.xlsx	final-model.pt	 testkoor.txt	  trainkomm.txt
corpus_rev_cleaned.xlsx		loss.tsv	 test_sample.csv  trainkoor.txt
deveff.txt			sample_data	 test.tsv	  train.txt
devform.txt			stopwordsid.txt  test.txt	  weights.txt


In [32]:
dfcred = df.loc[df['credibility']==1]
dfcred.head()
dfcred['formulasi'] = dfcred['formulasi'].replace({'1':1,'X':0,-1:2,'-1':2,0:2,'0':2,'-':0})
dfcred['efektivitas'] = dfcred['efektivitas'].replace({'1':1,'X':0,-1:2,'-1':2,0:2,'0':2,'-':0})
dfcred['koordinasi'] = dfcred['koordinasi'].replace({'1':1,'X':0,-1:2,'-1':2,0:2,'0':2,'-':0})
dfcred['stance komunikasi'] = dfcred['stance komunikasi'].replace({'1':1,'X':0,-1:2,'-1':2,0:3,'0':3,'-':0})
set(dfcred['formulasi'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

{0, 1, 2}

In [39]:
dftestcred = dftest.loc[dftest['credibility']==1]
dftestcred = dftestcred.dropna()
dftestcred['formulasi'] = dftestcred['formulasi'].replace({'1':1,'X':0,-1:2,'-1':2,0:2,'0':2,'-':0})
dftestcred['efektivitas'] = dftestcred['efektivitas'].replace({'1':1,'X':0,-1:2,'-1':2,0:2,'0':2,'-':0})
dftestcred['koordinasi'] = dftestcred['koordinasi'].replace({'1':1,'X':0,-1:2,'-1':2,0:2,'0':2,'-':0})
dftestcred['stance komunikasi'] = dftestcred['stance komunikasi'].replace({'1':1,'X':0,-1:2,'-1':2,0:3,'0':3,'-':0})
print(len(dftestcred))

257


In [0]:
import os
import math
import random
# os.remove("trainform.txt")
# os.remove("testform.txt")
# os.remove("devform.txt")

listrand = list(range(100))
random.shuffle(listrand)
listranddev = []
for i in range(math.floor(len(listrand)*0.2)):
    listranddev.append(listrand.pop())
file1 = open("trainform.txt","a", encoding="utf-8")
for i in listrand:
    file1.writelines(['__label__',str(dfcred['formulasi'].iloc[i]),' ',str(dfcred['sentence'].iloc[i]),'\n'])
file1.close()
file2 = open("devform.txt","a", encoding="utf-8")
for i in listranddev:
    file2.writelines(['__label__',str(dfcred['formulasi'].iloc[i]),' ',str(dfcred['sentence'].iloc[i]),'\n'])
file2.close()
file3 = open("testform.txt","a", encoding="utf-8")
for i in range(len(dftestcred)):
    file3.writelines(['__label__',str(dftestcred['formulasi'].iloc[i]),' ',str(dftestcred['sentence'].iloc[i]),'\n'])
file3.close()

In [53]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='testform.txt', dev_file='devform.txt', train_file='trainform.txt')

word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('id-forward'), FlairEmbeddings('id-backward')]

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=256, bidirectional=True,rnn_type='lstm',rnn_layers=1,dropout=0.2)

classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)

trainer = ModelTrainer(classifier, corpus)

trainer.train('./', max_epochs=30,checkpoint=True)
!mv best-model.pt best-form-model.pt

2019-07-17 14:01:26,730 Reading data from .
2019-07-17 14:01:26,733 Train: trainform.txt
2019-07-17 14:01:26,734 Dev: devform.txt
2019-07-17 14:01:26,735 Test: testform.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  import sys
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:463: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since versi

2019-07-17 14:01:29,648 Computing label dictionary. Progress:


100%|██████████| 80/80 [00:00<00:00, 35409.91it/s]

2019-07-17 14:01:29,657 [b'0', b'1', b'2']


2019-07-17 14:01:29,876 ----------------------------------------------------------------------------------------------------
2019-07-17 14:01:29,877 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.1)
          (encoder): Embedding(8823, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=8823, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.1)
          (encoder): Embedding(8823, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=8823, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=4196, out_features=4196, bias=True)
    (rnn): GRU(4196, 256, bidirectional=True)
    (dro

In [54]:
!ls

adc.json			devkomm.txt	 testform.txt	  training.log
best-cred-model.pt		devkoor.txt	 testkomm.txt	  trainkomm.txt
best-form-model.pt		dev.txt		 testkoor.txt	  trainkoor.txt
checkpoint.pt			final-model.pt	 test_sample.csv  train.txt
corpus_rev_cleaned_sample.xlsx	loss.tsv	 test.tsv	  weights.txt
corpus_rev_cleaned.xlsx		sample_data	 test.txt
deveff.txt			stopwordsid.txt  traineff.txt
devform.txt			testeff.txt	 trainform.txt


In [0]:
import os
import math
import random
# os.remove("traineff.txt")
# os.remove("testeff.txt")
# os.remove("deveff.txt")

listrand = list(range(100))
random.shuffle(listrand)
listranddev = []

for i in range(math.floor(len(listrand)*0.2)):
    listranddev.append(listrand.pop())
file1 = open("traineff.txt","a", encoding="utf-8")
for i in listrand:
    file1.writelines(['__label__',str(dfcred['efektivitas'].iloc[i]),' ',str(dfcred['sentence'].iloc[i]),'\n'])
file1.close()
file2 = open("deveff.txt","a", encoding="utf-8")
for i in listranddev:
    file2.writelines(['__label__',str(dfcred['efektivitas'].iloc[i]),' ',str(dfcred['sentence'].iloc[i]),'\n'])
file2.close()
file3 = open("testeff.txt","a", encoding="utf-8")
for i in range(len(dftestcred)):
    file3.writelines(['__label__',str(dftestcred['efektivitas'].iloc[i]),' ',str(dftestcred['sentence'].iloc[i]),'\n'])
file3.close()

In [55]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='testeff.txt', dev_file='deveff.txt', train_file='traineff.txt')

word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('id-forward'), FlairEmbeddings('id-backward')]

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=256, bidirectional=True,rnn_type='lstm',rnn_layers=1,dropout=0.2)

classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)

trainer = ModelTrainer(classifier, corpus)

trainer.train('./', max_epochs=30,checkpoint=True)
!mv best-model.pt best-eff-model.pt

2019-07-17 14:08:01,992 Reading data from .
2019-07-17 14:08:01,993 Train: traineff.txt
2019-07-17 14:08:01,995 Dev: deveff.txt
2019-07-17 14:08:01,998 Test: testeff.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  import sys
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:463: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since versi

2019-07-17 14:08:05,267 Computing label dictionary. Progress:


100%|██████████| 80/80 [00:00<00:00, 28469.74it/s]

2019-07-17 14:08:05,276 [b'1', b'0', b'2']


2019-07-17 14:08:05,513 ----------------------------------------------------------------------------------------------------
2019-07-17 14:08:05,516 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.1)
          (encoder): Embedding(8823, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=8823, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.1)
          (encoder): Embedding(8823, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=8823, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=4196, out_features=4196, bias=True)
    (rnn): GRU(4196, 256, bidirectional=True)
    (dro

In [0]:
import os
import math
import random
# os.remove("trainkoor.txt")
# os.remove("testkoor.txt")
# os.remove("devkoor.txt")

listrand = list(range(100))
random.shuffle(listrand)
listranddev = []

for i in range(math.floor(len(listrand)*0.2)):
    listranddev.append(listrand.pop())

file1 = open("trainkoor.txt","a", encoding="utf-8")
for i in listrand:
    file1.writelines(['__label__',str(dfcred['koordinasi'].iloc[i]),' ',str(dfcred['sentence'].iloc[i]),'\n'])
file1.close()
file2 = open("devkoor.txt","a", encoding="utf-8")
for i in listranddev:
    file2.writelines(['__label__',str(dfcred['koordinasi'].iloc[i]),' ',str(dfcred['sentence'].iloc[i]),'\n'])
file2.close()
file3 = open("testkoor.txt","a", encoding="utf-8")
for i in range(len(dftestcred)):
    file3.writelines(['__label__',str(dftestcred['koordinasi'].iloc[i]),' ',str(dftestcred['sentence'].iloc[i]),'\n'])
file3.close()

In [46]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='testkoor.txt', dev_file='devkoor.txt', train_file='trainkoor.txt')

word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('id-forward'), FlairEmbeddings('id-backward')]

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=256, bidirectional=True,rnn_type='lstm',rnn_layers=1,dropout=0.2)

classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)

trainer = ModelTrainer(classifier, corpus)

trainer.train('./', max_epochs=30,checkpoint=True)
!mv best-model.pt best-koor-model.pt

2019-07-17 13:42:13,492 Reading data from .
2019-07-17 13:42:13,494 Train: trainkoor.txt
2019-07-17 13:42:13,496 Dev: devkoor.txt
2019-07-17 13:42:13,497 Test: testkoor.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  import sys
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:463: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since versi

2019-07-17 13:42:16,124 Computing label dictionary. Progress:


100%|██████████| 80/80 [00:00<00:00, 44804.96it/s]

2019-07-17 13:42:16,135 [b'0', b'1', b'2']
2019-07-17 13:42:16,290 ----------------------------------------------------------------------------------------------------
2019-07-17 13:42:16,291 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.1)
          (encoder): Embedding(8823, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=8823, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.1)
          (encoder): Embedding(8823, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=8823, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=4196, out_features=4196, bias=True)
    (rnn): 

2019-07-17 13:42:16,980 epoch 1 - iter 0/3 - loss 1.12953818
2019-07-17 13:42:17,505 epoch 1 - iter 1/3 - loss 0.88430059
2019-07-17 13:42:17,838 epoch 1 - iter 2/3 - loss 0.68384539
2019-07-17 13:42:17,860 ----------------------------------------------------------------------------------------------------
2019-07-17 13:42:17,861 EPOCH 1 done: loss 0.6838 - lr 0.1000
2019-07-17 13:42:18,233 DEV : loss 0.2169550657272339 - score 0.95
2019-07-17 13:42:18,265 BAD EPOCHS (no improvement): 0
2019-07-17 13:42:29,995 ----------------------------------------------------------------------------------------------------
2019-07-17 13:42:30,097 epoch 2 - iter 0/3 - loss 0.24342805
2019-07-17 13:42:30,217 epoch 2 - iter 1/3 - loss 0.25471178
2019-07-17 13:42:30,266 epoch 2 - iter 2/3 - loss 0.23017298
2019-07-17 13:42:30,284 ----------------------------------------------------------------------------------------------------
2019-07-17 13:42:30,285 EPOCH 2 done: loss 0.2302 - lr 0.1000
2019-07-17 13

{'dev_loss_history': [tensor(0.2170, device='cuda:0'),
  tensor(0.1935, device='cuda:0'),
  tensor(0.1995, device='cuda:0'),
  tensor(0.1775, device='cuda:0'),
  tensor(0.1816, device='cuda:0'),
  tensor(0.1788, device='cuda:0'),
  tensor(0.1795, device='cuda:0'),
  tensor(0.1802, device='cuda:0'),
  tensor(0.1717, device='cuda:0'),
  tensor(0.1731, device='cuda:0'),
  tensor(0.1746, device='cuda:0'),
  tensor(0.1691, device='cuda:0'),
  tensor(0.1697, device='cuda:0'),
  tensor(0.1706, device='cuda:0'),
  tensor(0.1711, device='cuda:0'),
  tensor(0.1718, device='cuda:0'),
  tensor(0.1720, device='cuda:0'),
  tensor(0.1724, device='cuda:0'),
  tensor(0.1718, device='cuda:0'),
  tensor(0.1720, device='cuda:0'),
  tensor(0.1724, device='cuda:0'),
  tensor(0.1725, device='cuda:0'),
  tensor(0.1726, device='cuda:0'),
  tensor(0.1724, device='cuda:0'),
  tensor(0.1721, device='cuda:0'),
  tensor(0.1720, device='cuda:0'),
  tensor(0.1721, device='cuda:0'),
  tensor(0.1722, device='cuda:0'),


In [0]:
import os
import math
import random
# os.remove("trainkomm.txt")
# os.remove("testkomm.txt")
# os.remove("devkomm.txt")

listrand = list(range(100))
random.shuffle(listrand)
listranddev = []

for i in range(math.floor(len(listrand)*0.2)):
    listranddev.append(listrand.pop())

file1 = open("trainkomm.txt","a", encoding="utf-8")
for i in listrand:
    file1.writelines(['__label__',str(dfcred['stance komunikasi'].iloc[i]),' ',str(dfcred['sentence'].iloc[i]),'\n'])
file1.close()
file2 = open("devkomm.txt","a", encoding="utf-8")
for i in listranddev:
    file2.writelines(['__label__',str(dfcred['stance komunikasi'].iloc[i]),' ',str(dfcred['sentence'].iloc[i]),'\n'])
file2.close()
file3 = open("testkomm.txt","a", encoding="utf-8")
for i in range(len(dftestcred)):
    file3.writelines(['__label__',str(dftestcred['stance komunikasi'].iloc[i]),' ',str(dftestcred['sentence'].iloc[i]),'\n'])
file3.close()

In [48]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='testkomm.txt', dev_file='devkomm.txt', train_file='trainkomm.txt')

word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('id-forward'), FlairEmbeddings('id-backward')]

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=256, bidirectional=True,rnn_type='lstm',rnn_layers=1,dropout=0.2)

classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)

trainer = ModelTrainer(classifier, corpus)

trainer.train('./', max_epochs=30,checkpoint=True)
!mv best-model.pt best-komm-model.pt

2019-07-17 13:49:22,888 Reading data from .
2019-07-17 13:49:22,890 Train: trainkomm.txt
2019-07-17 13:49:22,891 Dev: devkomm.txt
2019-07-17 13:49:22,893 Test: testkomm.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  import sys
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:463: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since versi

2019-07-17 13:49:25,630 Computing label dictionary. Progress:


100%|██████████| 80/80 [00:00<00:00, 31167.04it/s]

2019-07-17 13:49:25,640 [b'2', b'0', b'3', b'1']
2019-07-17 13:49:25,808 ----------------------------------------------------------------------------------------------------
2019-07-17 13:49:25,809 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.1)
          (encoder): Embedding(8823, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=8823, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.1)
          (encoder): Embedding(8823, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=8823, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=4196, out_features=4196, bias=True)
    (

2019-07-17 13:49:25,835 Device: cuda:0
2019-07-17 13:49:25,836 ----------------------------------------------------------------------------------------------------
2019-07-17 13:49:25,837 Embedding storage mode: cpu
2019-07-17 13:49:25,840 ----------------------------------------------------------------------------------------------------
2019-07-17 13:49:26,482 epoch 1 - iter 0/3 - loss 1.39324415
2019-07-17 13:49:27,066 epoch 1 - iter 1/3 - loss 1.33005446
2019-07-17 13:49:27,458 epoch 1 - iter 2/3 - loss 1.25697132
2019-07-17 13:49:27,476 ----------------------------------------------------------------------------------------------------
2019-07-17 13:49:27,477 EPOCH 1 done: loss 1.2570 - lr 0.1000
2019-07-17 13:49:27,755 DEV : loss 0.8324764370918274 - score 0.85
2019-07-17 13:49:27,783 BAD EPOCHS (no improvement): 0
2019-07-17 13:49:39,777 ----------------------------------------------------------------------------------------------------
2019-07-17 13:49:39,884 epoch 2 - iter 0/3

{'dev_loss_history': [tensor(0.8325, device='cuda:0'),
  tensor(0.6135, device='cuda:0'),
  tensor(0.6988, device='cuda:0'),
  tensor(0.8798, device='cuda:0'),
  tensor(0.7778, device='cuda:0'),
  tensor(0.8207, device='cuda:0'),
  tensor(0.8023, device='cuda:0'),
  tensor(0.8198, device='cuda:0'),
  tensor(0.8409, device='cuda:0'),
  tensor(0.8397, device='cuda:0'),
  tensor(0.8473, device='cuda:0'),
  tensor(0.8534, device='cuda:0'),
  tensor(0.8608, device='cuda:0'),
  tensor(0.8621, device='cuda:0'),
  tensor(0.8674, device='cuda:0'),
  tensor(0.8692, device='cuda:0'),
  tensor(0.8710, device='cuda:0'),
  tensor(0.8717, device='cuda:0'),
  tensor(0.8719, device='cuda:0'),
  tensor(0.8734, device='cuda:0'),
  tensor(0.8738, device='cuda:0'),
  tensor(0.8741, device='cuda:0'),
  tensor(0.8742, device='cuda:0'),
  tensor(0.8759, device='cuda:0'),
  tensor(0.8767, device='cuda:0'),
  tensor(0.8771, device='cuda:0'),
  tensor(0.8774, device='cuda:0'),
  tensor(0.8780, device='cuda:0'),


In [49]:
uploaded = drive.CreateFile({'title': 'best-cred-model.pt'})
uploaded.SetContentFile('best-cred-model.pt')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))
uploaded = drive.CreateFile({'title': 'best-form-model.pt'})

uploaded.SetContentFile('best-form-model.pt')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))
uploaded = drive.CreateFile({'title': 'best-form-model.pt'})

uploaded.SetContentFile('best-eff-model.pt')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))
uploaded = drive.CreateFile({'title': 'best-eff-model.pt'})

adc.json			devkomm.txt	 testeff.txt	  traineff.txt
best-model.pt			devkoor.txt	 testform.txt	  trainform.txt
checkpoint.pt			dev.txt		 testkomm.txt	  training.log
corpus_rev_cleaned_sample.xlsx	final-model.pt	 testkoor.txt	  trainkomm.txt
corpus_rev_cleaned.xlsx		loss.tsv	 test_sample.csv  trainkoor.txt
deveff.txt			sample_data	 test.tsv	  train.txt
devform.txt			stopwordsid.txt  test.txt	  weights.txt
